In [63]:
from numpy import array
from pickle import dump
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding

from numpy import array
from numpy import asarray
from numpy import zeros
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Flatten

from sklearn.preprocessing import OneHotEncoder
import gensim 
from gensim.models import Word2Vec

In [2]:
# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

In [3]:
# load
in_filename = 'republic_sequences.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')

In [4]:
# integer encode sequences of words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)

In [5]:
print(doc)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [6]:
# vocabulary size
vocab_size = len(tokenizer.word_index) + 1

## embedding style

#### one hot encoding

In [7]:
# separate into input and output
sequences = array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]

In [33]:
print(X.shape)

(219583, 50)


In [34]:
print(y.shape)

(219583, 10650)


### Word Vector

#### Glove encoding

In [41]:
# pad documents to a max length of 4 words
max_length = 4
padded_docs = pad_sequences(sequences, maxlen=max_length, padding='post')
print(padded_docs)
# load the whole embedding into memory
embeddings_index = dict()
f = open('./glove.6B/glove.6B.300d.txt', encoding='utf8')
for line in f:
	values = line.split()
	word = values[0]
	coefs = asarray(values[1:], dtype='float32')
	embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))
# create a weight matrix for words in training docs
embedding_matrix = zeros((vocab_size, 300))
for word, i in tokenizer.word_index.items():
	embedding_vector = embeddings_index.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector

[[   30  1846    13  3276]
 [ 1846    13  3276    35]
 [   13  3276    35  3887]
 ...
 [ 3886 10649     4   320]
 [10649     4   320    85]
 [    4   320    85   314]]
Loaded 400000 word vectors.


In [47]:
Glove_model = Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=50, trainable=False)
# evaluate the model
# loss, accuracy = model.evaluate(padded_docs, labels, verbose=0)
# print('Accuracy: %f' % (accuracy*100))

In [ ]:
# print(Glove_model.shape)

#### CBOW embedding

In [54]:
model_cbow = gensim.models.Word2Vec(tokenizer.word_index, min_count = 1,  
                              size = 100, window = 5) 

In [55]:
print(model_cbow.syn1neg)

[[-0.05561198 -0.21587288  0.20985548 ...  0.02114867 -0.01400216
   0.04276157]
 [-0.05186394 -0.19816124  0.17257296 ...  0.01430864 -0.02946857
   0.04394772]
 [-0.04030392 -0.2008115   0.1779106  ...  0.02733215 -0.07335254
   0.01314793]
 ...
 [-0.01435178 -0.06815012  0.06055047 ...  0.00525459 -0.02738453
   0.0041643 ]
 [-0.01351258 -0.0728265   0.05699909 ...  0.00688104 -0.01846179
   0.00933912]
 [-0.01928682 -0.09713058  0.08563205 ...  0.01177776 -0.02991945
   0.01039469]]


<ipython-input-55-f3da59bb6fa3>:1: DeprecationWarning: Call to deprecated `syn1neg` (Attribute will be removed in 4.0.0, use self.trainables.syn1neg instead).
  print(model_cbow.syn1neg)


In [75]:
gensim.models.Word2Vec.save_word2vec_format(fname='./model_cbow.txt', binary=False)
# model_cbow.save('../model_cbow')

TypeError: save_word2vec_format() missing 1 required positional argument: 'self'

In [50]:
# define model
model = Sequential()
# keras embedding
# model.add(Embedding(vocab_size, 50, weight= model_cbow.syn1neg, input_length=seq_length))

# cbow embedding
model.add(Glove_model)

model.add(LSTM(300, return_sequences=True))
model.add(LSTM(300))
model.add(Dense(300, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))

In [51]:
print(model.summary())

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 50, 300)           3195000   
_________________________________________________________________
lstm_7 (LSTM)                (None, 50, 300)           721200    
_________________________________________________________________
lstm_8 (LSTM)                (None, 300)               721200    
_________________________________________________________________
dense_7 (Dense)              (None, 300)               90300     
_________________________________________________________________
dense_8 (Dense)              (None, 10650)             3205650   
Total params: 7,933,350
Trainable params: 4,738,350
Non-trainable params: 3,195,000
_________________________________________________________________
None


In [52]:
# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [53]:
# fit model
model.fit(X, y, batch_size=128, epochs=100)
# fit model glove
# model.fit(padded_docs, max_length, batch_size=128, epochs=100)

Epoch 1/100
   768/219583 [..............................] - ETA: 36:24 - loss: 9.0613 - accuracy: 0.0495  

KeyboardInterrupt: 

In [ ]:
# save the model to file
model.save('model.h5')

In [ ]:
# save the tokenizer
dump(tokenizer, open('tokenizer.pkl', 'wb'))